# Modelos y Simulación 2024

## Trabajo Práctico Final:
## Hospital

### Alumnos:
- Ana Enzo
- Barea Matias
- Irigoyen Carlos
- Morillo Alen

### Profesores: Lic. Claudia López de Munain - Lic. Bruno Alfredo Pazos

In [1]:
from hospital import *

In [55]:
tiempo_cirugia = determinar_tiempo_operacion()
op_diarias = generar_cantidad_cirugias_diarias()
cantidad = np.random.poisson(tiempo_cirugia, op_diarias )
print(tiempo_cirugia)
print(cantidad)
print(np.sum(cantidad))

2.717715285211587
[5 4 5 3 2 2 1 4 1 2]


In [2]:
''' TEST SIMULACION'''
simulacion(2,210,4,12,140,4)

Cantidad de quirofanos: 4 
 Kits iniciales: 140 
 llegada de pacientes por dia: [118, 114, 86, 100, 104, 88, 112, 120, 126, 96, 121, 112, 91, 104, 121, 116, 107, 123, 101, 108, 124, 101, 99, 108, 111, 104, 117, 115, 110, 108, 115, 110, 114, 116, 119, 106, 100, 107, 118, 115, 114, 101, 90, 102, 117, 95, 114, 103, 107, 116, 134, 101, 108, 104, 105, 121, 117, 113, 108, 109, 107, 117, 108, 118, 122, 109, 110, 102, 106, 84, 117, 120, 104, 101, 123, 119, 116, 113, 92, 98, 103, 119, 128, 100, 90, 109, 111, 131, 128, 116, 110, 98, 106, 106, 104, 102, 101, 94, 121, 112, 106, 94, 127, 106, 99, 100, 117, 110, 106, 106, 101, 103, 106, 110, 103, 108, 104, 111, 108, 116, 121, 123, 97, 123, 126, 108, 111, 111, 91, 98, 124, 104, 115, 98, 110, 118, 118, 111, 95, 112, 100, 113, 110, 111, 122, 109, 121, 109, 113, 121, 128, 113, 123, 119, 102, 124, 118, 85, 108, 97, 122, 120, 103, 108, 117, 82, 109, 110, 104, 112, 110, 106, 109, 113, 116, 113, 98, 107, 107, 112, 112, 102, 97, 100, 105, 104, 112, 105, 102,

In [15]:
import numpy as np
import matplotlib.pyplot as plt

# Tiempo simulacion
ANIOS_SIMULACION = 2
DIAS_SIMULACION = 365 * ANIOS_SIMULACION # 730 dias

# Valores promedios
PROMEDIO_KITS_MENSUALES = 130 # sigue una distribucion de poisson
MEDIA_PACIENTES_DIARIOS = 110 # sigue una distribucion de poisson
PROMEDIO_TIEMPO_CIRUGIA = 1.5 # tiempo en horas
PROB_RESERVA_QUIRONAFO = 0.43 # 43% Uniforme
PROMEDIO_TIEMPO_INTERNACION = 2  # dias, sigue una distribucion exponencial
PROMEDIO_CIRUGIAS_DIARIAS = 8 # por quirofano

# Parametros iniciales de la simulacion
KITS_INICIALES = np.random.poisson(PROMEDIO_KITS_MENSUALES)
CAMAS_TOTALES = 210 # parametro de entrada para comprarar con los objetivos propuestos
CANT_QUIROFANOS = 4 # parametro de entrada para comparar con los objetivos propuestos
CANT_HORAS_ATENCION_QUIROFANO = 12 # tiempo en horas
KITS_REPOSICION_DIARIA = 4 # parametro de entrada para comprarar con los objetivos propuestos

# Paciente. Atributos: DIA DE INTERNACION Y DIA DE ALTA
paciente_base = {
    'dia_internacion': 0,
    'dia_alta': 0,
}

# Arrays
# disponibilidad_camas = [CAMAS_TOTALES] * DIAS_SIMULACION

# Calcular probabilidades
def generar_demanda_diaria():
    return np.random.poisson(MEDIA_PACIENTES_DIARIOS) # CANTIDAD DE PACIENTES QUE INGRESAN DIARIAMENTE

def generar_dias_internacion():
    return np.random.exponential(PROMEDIO_TIEMPO_INTERNACION) # TIEMPO EN EL QUE LOS PACIENTES ESTAN INTERNADOS OCUPANDO UNA CAMA

def generar_cantidad_cirugias_diarias():
    return np.random.poisson(PROMEDIO_CIRUGIAS_DIARIAS)

def determinar_tiempo_operacion():
    return np.random.exponential(PROMEDIO_TIEMPO_CIRUGIA) # TIEMPO DE SALIDA DE PACIENTE DEL QUIROFANO

def asignar_cama(estado_sistema, dia, cant_dias_internacion):

    for dia_estadia in range(dia, dia+cant_dias_internacion):
        estado_sistema['disponibilidad_camas'][dia_estadia] -= 1

def crear_paciente(dia_internacion, dia_alta):
    paciente = paciente_base.copy()
    paciente['dia_internacion'] = dia_internacion
    paciente['dia_alta'] = dia_alta
    return paciente

def asignar_quirofano(paciente,quirofanos, quirofano):
    quirofanos[quirofano].append(paciente)

def procesar_pacientes(estado_sistema, dia, quirofanos, cantidad_pacientes):

    quirofano_counter = 0
    quirofano_keys = list(quirofanos.keys())
    num_quirofanos = len(quirofanos)
    pacientes_sin_reserva = 0
    pacientes_con_reserva = 0

    for i in range(cantidad_pacientes):               
        cant_dias_internacion = generar_dias_internacion()
        cant_dias_internacion = 1 if cant_dias_internacion < 1 else cant_dias_internacion # Tomamos los pacientes < 0 dias internacion como = 1(un) dia de internacion
        dias_int = int(cant_dias_internacion)
        if dia+dias_int <= DIAS_SIMULACION:
            asignar_cama(estado_sistema, dia, dias_int)
            necesita_quirofano = np.random.uniform(0,1)
            if necesita_quirofano >= 0.43:
                paciente = crear_paciente(dia, dia+cant_dias_internacion)
                quirofano = quirofano_keys[quirofano_counter % num_quirofanos]
                asignar_quirofano(paciente,quirofanos, quirofano)
                quirofano_counter += 1
                pacientes_con_reserva += 1
            else:
                pacientes_sin_reserva += 1
                
        else:
            break
    estado_sistema['reservas_por_dia'].append(pacientes_con_reserva)
    estado_sistema['pacientes_sin_reserva_por_dia'].append(pacientes_sin_reserva)    

def hay_kits(kits_disponibles):
    return kits_disponibles > 0

def inicializar_quirofanos(cantidad_quirofanos):
    quirofanos = {f'quirofano {i+1}': [] for i in range(cantidad_quirofanos)}
    ocupacion_quirofanos = {f'quirofano {i+1}': 0 for i in range(cantidad_quirofanos)}
    return quirofanos, ocupacion_quirofanos

def simulacion():

    # Contadores globales totales

    estado_sistema = {

        'cantidad de quirofanos:': 0,
        'kits_iniciales': 0,
        'llegadas_por_dia': [],
        'reservas_por_dia': [],
        'disponibilidad_camas': [CAMAS_TOTALES] * DIAS_SIMULACION,

        'pacientes_rechazados_por_dia': [],
        'pacientes_sin_reserva_por_dia': [], 
        'kits_diarios_utilizados': [],
        'ocupacion_diaria_quirofanos': [],
        'tiempo_espera_diario_quirofanos': [],
        'cirugias_rechazadas_por_dia' : [],
        'cirugias_concretadas_por_dia' : [],
        'cirugias_reprogramadas_por_insumos': [],
        'cirugias_reprogramadas_por_tiempo': [],

        'total_cirugias_reprogramadas_por_insumos': 0,
        'total_cirugias_reprogramadas_por_tiempo': 0,
        'total_cirugias_concretadas': 0,
        'total_cirugias_rechazadas': 0,
        'total_pacientes_rechazados': 0

    }

    cirugias_reprogramadas_por_insumos = 0
    cirugias_reprogramadas_por_tiempo = 0
    cirugias_concretadas = 0
    cirugias_rechazadas = 0
    pacientes_rechazados = 0

    estado_sistema['kits_iniciales'] = KITS_INICIALES
    
    kits_disponibles = estado_sistema['kits_iniciales']

    
    quirofanos, ocupacion_quirofanos = inicializar_quirofanos(CANT_QUIROFANOS)
    
    for dia in range(DIAS_SIMULACION):
        
        ocupacion_quirofanos = {key: 0 for key in ocupacion_quirofanos}

        kits_disponibles += KITS_REPOSICION_DIARIA #Incrementar kits diariamente
        
        '''ORGANIZACION DE LA DEMANDA'''

        llegadas = generar_demanda_diaria()

        estado_sistema['llegadas_por_dia'].append(llegadas)

        if estado_sistema['disponibilidad_camas'][dia] >= llegadas:
            procesar_pacientes(estado_sistema, dia, quirofanos, llegadas)
        else:
            pacientes_rechazados += llegadas - estado_sistema['disponibilidad_camas'][dia]
            camas_disponibles = estado_sistema['disponibilidad_camas'][dia]
            procesar_pacientes(estado_sistema, dia, quirofanos, camas_disponibles)

        '''INICIO HORARIO ATENCION'''
       
        for key in quirofanos:
            while (ocupacion_quirofanos[key] <= CANT_HORAS_ATENCION_QUIROFANO and len(quirofanos[key]) > 0 and hay_kits(kits_disponibles)):
                paciente = quirofanos[key].pop(0)
                if dia <= paciente['dia_alta']:
                    if hay_kits(kits_disponibles):
                        ocupacion_quirofanos[key] += determinar_tiempo_operacion()
                        kits_disponibles -= 1
                        cirugias_concretadas += 1
                        estado_sistema['tiempo_espera_diario_quirofanos'].append(dia - paciente['dia_internacion']) # El tiempo que esperan los pacientes para ingresar al quirofano
                    else:
                        cirugias_reprogramadas_por_insumos += 1
                        quirofanos[key].insert(0, paciente)
  
                else:
                    pacientes_rechazados += 1
                    cirugias_rechazadas += 1
            if kits_disponibles == 0:
                cirugias_reprogramadas_por_insumos += len(quirofanos[key])
            else:
                cirugias_reprogramadas_por_tiempo += len(quirofanos[key])      

        # Contadores que son por dia.
        estado_sistema['kits_diarios_utilizados'].append(kits_disponibles)
        estado_sistema['ocupacion_diaria_quirofanos'].append(ocupacion_quirofanos)
        estado_sistema['pacientes_rechazados_por_dia'].append(pacientes_rechazados)
        estado_sistema['cirugias_rechazadas_por_dia'].append(cirugias_rechazadas)
        estado_sistema['cirugias_concretadas_por_dia'].append(cirugias_concretadas)
        estado_sistema['cirugias_reprogramadas_por_insumos'].append(cirugias_reprogramadas_por_insumos)
        estado_sistema['cirugias_reprogramadas_por_tiempo'].append(cirugias_reprogramadas_por_tiempo)
    
    # Contadores totales
    estado_sistema['total_cirugias_reprogramadas_por_insumos'] += cirugias_reprogramadas_por_insumos
    estado_sistema['total_cirugias_reprogramadas_por_tiempo'] += cirugias_reprogramadas_por_tiempo
    estado_sistema['total_cirugias_concretadas'] += cirugias_concretadas
    estado_sistema['total_cirugias_rechazadas'] += cirugias_rechazadas
    estado_sistema['total_pacientes_rechazados'] += pacientes_rechazados
    
    print(f'Cantidad de quirofanos:', estado_sistema['cantidad de quirofanos:'],'\n',
          'Kits iniciales:', estado_sistema['kits_iniciales'], '\n',
          'llegada de pacientes por dia:', estado_sistema['llegadas_por_dia'], '\n',
          'reservas de quirofano por dia:', estado_sistema['reservas_por_dia'], '\n',
          'disponibilidad de camas por dia:', estado_sistema['disponibilidad_camas'], '\n',
          'kits utilizados por dia:', estado_sistema['kits_diarios_utilizados'], '\n',
          'ocupacion diaria quirofanos (en horas):', estado_sistema['ocupacion_diaria_quirofanos'], '\n',
          'cuanto esperan los pacientes por dia en ser operados:', estado_sistema['tiempo_espera_diario_quirofanos'] , '\n',
          'pacientes que solo se internaron:', estado_sistema['pacientes_sin_reserva_por_dia'], '\n',
          'pacientes rechazados por dia:', estado_sistema['pacientes_rechazados_por_dia'], '\n',
          'cirugias concretadas por dia:', estado_sistema['cirugias_concretadas_por_dia'], '\n',
          'cirugias diarias reprogramadas por falta de insumos:', estado_sistema['cirugias_reprogramadas_por_insumos'], '\n',
          'cirugias diaria reprogramadas al otro dia por ocupacion del quirofano:', estado_sistema['cirugias_reprogramadas_por_tiempo'], '\n',
          'Total de cirugias reprogramadas por falta insumos:', estado_sistema['total_cirugias_reprogramadas_por_insumos'], '\n',
          'Total de cirugias reprogramadas por falta de tiempo:', estado_sistema['total_cirugias_reprogramadas_por_tiempo'], '\n',
          'Total cirugias concretadas:', estado_sistema['total_cirugias_concretadas'], '\n',
          'Total cirugias rechazadas:', estado_sistema['total_cirugias_rechazadas'], '\n',
          'Total pacientes rechazados:', estado_sistema['total_pacientes_rechazados']

          )
    


simulacion()

Cantidad de quirofanos: 0 
 Kits iniciales: 136 
 llegada de pacientes por dia: [112, 105, 130, 108, 96, 120, 112, 97, 112, 119, 103, 119, 112, 108, 97, 113, 128, 121, 118, 106, 92, 109, 108, 111, 113, 113, 111, 111, 108, 97, 113, 107, 141, 120, 114, 114, 111, 110, 115, 95, 99, 111, 105, 106, 134, 111, 96, 113, 99, 105, 111, 108, 115, 86, 123, 118, 105, 114, 108, 99, 101, 116, 129, 96, 101, 111, 107, 103, 110, 112, 123, 97, 122, 112, 93, 104, 101, 131, 106, 123, 100, 116, 110, 104, 111, 125, 108, 111, 122, 116, 103, 106, 102, 112, 120, 114, 126, 118, 110, 98, 115, 113, 101, 117, 116, 121, 122, 129, 121, 114, 126, 119, 107, 107, 115, 129, 120, 102, 113, 108, 116, 98, 97, 86, 82, 104, 108, 101, 110, 95, 110, 114, 138, 113, 131, 115, 107, 106, 109, 104, 114, 132, 110, 115, 105, 97, 113, 111, 111, 109, 108, 110, 119, 107, 120, 108, 108, 109, 115, 124, 119, 102, 103, 126, 136, 130, 109, 115, 114, 111, 131, 115, 142, 108, 122, 104, 99, 111, 116, 106, 114, 113, 109, 115, 104, 125, 95, 98, 109